This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets. In addition, it also needs an OpenAI token as a secret with key `openai_key`

In [3]:
pip install neo4j-graphrag neo4j openai yfiles_jupyter_graphs_for_neo4j rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.9 MB/s eta 0:00:00


In [4]:
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j_graphrag.indexes import create_vector_index
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')
neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

articles = [
    {
        "title": "Tesla's Q3 2024 Earnings: A Surprise Beat",
        "content": "Tesla reported record revenue in Q3 2024, exceeding analysts' expectations due to increased Model 3 and Model Y sales."
    },
    {
        "title": "Analysts Upgrade Tesla Stock Following Strong Performance",
        "content": "Several analysts upgraded their ratings on Tesla stock after the Q3 earnings report, citing strong demand for electric vehicles."
    },
    {
        "title": "Global Supply Chain Recovery Boosts Tesla Production",
        "content": "Improved supply chain conditions allowed Tesla to ramp up production, contributing significantly to its stock price surge."
    },
    {
        "title": "Tesla's Innovative Battery Technology Enhances Range",
        "content": "Recent breakthroughs in battery technology have enabled Tesla to improve the range of its electric vehicles significantly."
    },
    {
        "title": "Market Trends Show Growing Demand for EVs",
        "content": "Overall market trends indicate a surge in consumer interest and demand for electric vehicles across multiple regions."
    }
]

ex_llm=LLM(
   model_name="gpt-4o-mini",
   model_params={
       "response_format": {"type": "json_object"},
       "temperature": 0
   })

embedder = Embeddings()

# 1. Build KG and Store in Neo4j Database
kg_builder = SimpleKGPipeline(
   llm=ex_llm,
   driver=driver,
   embedder=embedder,
   from_pdf=False
)
for article in articles:
    print(f"Processing: {article['title']}")
    result = await kg_builder.run_async(text=f"title: {article['title']} - content: {article['content']}")

create_vector_index(driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

# 2. KG Retriever
vector_retriever = VectorRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder
)

# 3. GraphRAG Class
llm = LLM(model_name="gpt-4o")
rag = GraphRAG(llm=llm, retriever=vector_retriever)

Processing: Tesla's Q3 2024 Earnings: A Surprise Beat
Processing: Analysts Upgrade Tesla Stock Following Strong Performance
Processing: Global Supply Chain Recovery Boosts Tesla Production
Processing: Tesla's Innovative Battery Technology Enhances Range
Processing: Market Trends Show Growing Demand for EVs


In [7]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Now let's perform some local search

In [8]:
response = rag.search("What were the key factors contributing to Tesla's strong performance in Q3 2024?")
print(response.answer)

Tesla's strong performance in Q3 2024 was driven by several key factors: record revenue due to increased sales of the Model 3 and Model Y, improved supply chain conditions that allowed for ramped-up production, and a wider market trend showing growing demand for electric vehicles. Additionally, innovations in battery technology improved the range of Tesla's electric vehicles, further boosting their appeal. Analysts also reacted positively, upgrading their ratings on Tesla stock following the strong earnings report.


In [9]:
response = rag.search("What was the contribution of Model Y to Tesla's bottom line?")
print(response.answer)

The context mentions that Tesla reported record revenue in Q3 2024, which exceeded analysts' expectations due to increased sales of Model 3 and Model Y. Therefore, the Model Y contributed significantly to Tesla's bottom line by being one of the key models driving record sales and revenue during this period.


Time for some global search

In [11]:
response = rag.search("How many facts do you have about Tesla? List them.")
print(response.answer)

I have five facts about Tesla:

1. Tesla's innovative battery technology has enhanced the range of its electric vehicles significantly.
2. Several analysts upgraded their ratings on Tesla stock following strong performance and demand for electric vehicles after the Q3 earnings report.
3. Tesla reported record revenue in Q3 2024, exceeding analysts' expectations due to increased sales of Model 3 and Model Y.
4. Improved global supply chain conditions have allowed Tesla to ramp up production, contributing significantly to a surge in its stock price.
5. Market trends show a growing demand for electric vehicles across multiple regions.


In [12]:
def clear_graph(tx):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(clear_graph)